In [34]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [35]:
conn = sqlite3.connect('db/RSS.sqlite')

In [36]:
df = pd.read_sql_query("SELECT * FROM RSS", conn)

In [37]:
df.groupby('publisher').size()

publisher
BAND      96
BBC       20
EBC       59
Folha    215
G1       711
R7       627
UOL      308
Valor    247
dtype: int64

In [38]:
df.groupby('keyword').size()

keyword
agro                     20
arte                     68
brasil                  156
carreira                 79
carros                   47
ciencia                  19
ciencia e saude          47
cinema                   58
comida                   17
economia                246
educacao                 93
empresas                 55
esporte                 347
geral                    96
loterias                 42
moda                     22
mundo                   246
musica                   51
natureza                 57
politica                 57
saude                    13
tecnologia              257
tecnologia e  games      40
tecnologia e ciencia     89
turismo                  61
dtype: int64